In [ ]:
import sounddevice as sd
import numpy as np
import threading
import time

# Fonction pour générer un signal de bruit blanc
def generate_white_noise(duration, sample_rate):
    samples = np.random.normal(0, 1, int(duration * sample_rate))
    return samples

# Fonction pour générer une sinusoïde de 440 Hz
def generate_sine_wave(duration, sample_rate):
    frequency = 440  # Hz
    samples = (np.sin(2 * np.pi * np.arange(sample_rate * duration) * frequency / sample_rate)).astype(np.float32)
    return samples

# Fonction pour lire le signal sur les haut-parleurs
def play_signal(signal, sample_rate):
    sd.play(signal, sample_rate)
    sd.wait()

# Fonction pour enregistrer le son du microphone
def record_microphone():
    duration = 5  # Durée de l'enregistrement en secondes
    sample_rate = 44100  # Fréquence d'échantillonnage en Hz

    # Demander à l'utilisateur de choisir le type de signal à générer
    signal_type = input("Entrez le type de signal à générer (blanc/sinus) : ")

    if signal_type == "blanc":
        signal = generate_white_noise(duration, sample_rate)
    elif signal_type == "sinus":
        signal = generate_sine_wave(duration, sample_rate)
    else:
        print("Type de signal non reconnu.")
        return

    # Démarrer un thread pour lire le signal sur les haut-parleurs
    play_thread = threading.Thread(target=play_signal, args=(signal, sample_rate))
    play_thread.start()

    # Enregistrer le son du microphone dans un fichier WAV
    filename = "enregistrement.wav"
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
    sd.wait()
    sd.write(filename, sample_rate, recording)

    # Attendre la fin de la lecture du signal sur les haut-parleurs
    play_thread.join()

# Démarrer un thread pour enregistrer le son du microphone et un autre thread pour jouer le signal en même temps
record_thread = threading.Thread(target=record_microphone)
play_thread = threading.Thread(target=play_signal, args=(np.zeros(44100 * 5), 44100)) # Thread pour jouer un silence de 5 secondes avant l'enregistrement
play_thread.start()
record_thread.start()

# Attendre la fin des deux threads
play_thread.join()
record_thread.join()
